In [21]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
prompt = ChatPromptTemplate.from_messages(
    [ ("system", "당신은 개발자입니다.") , 
     ("user", "{input}")]
)
print(prompt)

prompt_text = prompt.format(input="JavaScript는 무엇인가요? 자세하게 설명해주세요.")

input_variables=['input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 개발자입니다.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]


In [3]:
# Groq API를 사용하는 ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x000001F314225E50> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001F314296F60> root_client=<openai.OpenAI object at 0x000001F3114D4530> root_async_client=<openai.AsyncOpenAI object at 0x000001F313D9BC20> model_name='meta-llama/llama-4-scout-17b-16e-instruct' temperature=0.7 model_kwargs={} openai_api_key=SecretStr('**********') openai_api_base='https://api.groq.com/openai/v1'


In [4]:
try:
    response = llm.invoke(prompt_text)
    print(type(response))
    print(response)
    print("응답:", response.content)
except Exception as e:
    print(f"오류 발생: {e}")

오류 발생: Error code: 401 - {'error': {'message': 'Invalid API Key', 'type': 'invalid_request_error', 'code': 'invalid_api_key'}}


### LCEL (Prompt + LLM모델)

In [1]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("You are an expert in AI Expert. Answer the question. <Question>: {input}에 대해 쉽게 설명해주세요.")
prompt

PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='You are an expert in AI Expert. Answer the question. <Question>: {input}에 대해 쉽게 설명해주세요.')

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")

# chain 연결 (LCEL)
chain = prompt | llm

# chain 호출
result = chain.invoke({"input": "인공지능 모델의 학습 원리"})
print(type(result))
print(result.content)

<class 'langchain_core.messages.ai.AIMessage'>
인공지능 모델의 학습 원리는 크게 두 가지 방법으로 나눌 수 있습니다. 첫 번째는 지도학습이며, 이는 레이블이 달린 데이터를 사용하여 모델을 학습시키는 방법입니다. 모델은 입력 데이터와 해당 입력 데이터에 대한 정확한 출력(레이블)을 학습하여 패턴을 식별하고 예측을 수행합니다. 

두 번째 방법은 비지도학습입니다. 이는 레이블이 달려있지 않은 데이터를 사용하여 모델을 학습시키는 방법입니다. 모델은 입력 데이터의 패턴을 식별하고 클러스터링, 차원 축소, 이상치 탐지 등의 작업을 수행합니다.

두 방법 모두 학습 과정은 데이터를 입력하고 모델이 예측한 출력 값과 실제 값 사이의 오차를 최소화하는 방향으로 가중치를 조정하면서 반복적으로 학습이 진행됩니다. 이를 통해 모델은 데이터의 패턴을 학습하고 새로운 입력에 대한 예측을 수행할 수 있습니다.


### LLM 체인
- ( Prompt + LLM 모델 + 출력파서) - LCEL

In [5]:
# 1. 프롬프트 생성
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate.from_template(
    "You are an expert in AI Expert. Answer the question. <Question>: {input}에 대해 쉽게 설명해주세요."
)
prompt

PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='You are an expert in AI Expert. Answer the question. <Question>: {input}에 대해 쉽게 설명해주세요.')

In [6]:
llm = ChatOpenAI(model="gpt-3.5-turbo")

output_parser = StrOutputParser()

In [8]:
# 2. chain 생성 (LCEL)
chain = prompt | llm | output_parser

In [9]:
# 3. chain의 invoke 호출
result = chain.invoke({"input": "인공지능 모델의 학습 원리"})
print(type(result))
print(result)

<class 'str'>
<Answer>:
인공지능 모델의 학습원리는 데이터를 입력으로 받아 일련의 계산을 통해 예측을 수행하는 과정입니다. 모델은 입력 데이터와 정답 데이터 사이의 관계를 학습하여 새로운 데이터에 대해 예측을 할 수 있도록 학습됩니다. 학습 과정에서 모델은 오차를 최소화하기 위해 가중치를 조정하며, 이를 통해 데이터 간의 패턴과 관련성을 파악하게 됩니다. 이러한 학습과정을 통해 모델은 새로운 데이터에 대한 예측을 보다 정확하게 수행할 수 있게됩니다.


In [10]:
try:
    result = chain.invoke({"input": "LangChain의 Products(제품)는 어떤 것들이 있나요?"})
    print(type(result))
    print(result)
except Exception as e:
    print(f"오류 발생: {e}")

<class 'str'>
LangChain은 다양한 AI 기술과 서비스를 제공하는 기업입니다. LangChain의 제품에는 다국어 번역 및 통역 솔루션, 자연어 처리 기술을 활용한 챗봇 및 가상 비서 서비스, 언어 학습 및 교육 플랫폼 등이 있습니다. 또한 LangChain은 고객의 요구에 맞는 맞춤형 AI 솔루션을 개발하여 제공하고 있습니다.


### Runnable의 stream() 함수 호출

In [15]:
# 스트리밍 출력을 위한 요청
try:
    answer = chain.stream({"input": "인공지능 모델의 학습 원리를 자세하게 설명해 주세요."})
    # 스트리밍 출력
    print(answer)
except Exception as e:
    print(f"Error: {e}")

for token in answer:
    # 스트림에서 받은 데이터의 내용을 출력합니다. 줄바꿈 없이 이어서 출력하고, 버퍼를 즉시 비웁니다.
    print(token, end="", flush=True)

<generator object RunnableSequence.stream at 0x000001F30CB5F2E0>
인공지능 모델의 학습 원리를 쉽게 설명해 드리겠습니다.

인공지능 모델은 데이터를 입력받아 학습을 하고, 그 데이터에서 패턴을 찾아내어 새로운 데이터를 예측하거나 분류하는 작업을 수행합니다. 이 과정은 크게 입력 데이터를 받아들이고, 이를 가공하여 출력하는 과정으로 나눌 수 있습니다.

학습 과정은 입력 데이터를 모델에 주입하고, 모델이 주어진 데이터를 분석하여 일정한 패턴이나 특징을 학습합니다. 이때 모델은 사전에 정의된 목표에 맞게 최적화되는데, 이것을 어떤 기준 함수를 사용하여 측정할 수 있습니다.

모델이 학습을 통해 데이터를 분류하거나 예측할 수 있는 이유는 이렇게 학습된 패턴을 기반으로 새로운 데이터에 대한 예측을 수행할 수 있기 때문입니다.따라서, 인공지능 모델의 핵심은 데이터를 효과적으로 분석하여 패턴을 찾아내는 것이라고 할 수 있습니다.

### Multi Chain
* 첫 번째 Chain의 출력이, 두 번째 Chain의 입력이 된다.
* 두 개의 Chain과 Prompt + OutputParser를 LCEL로 연결하기

In [18]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Step 1: 사용자가 입력한 장르에 따라 영화 추천
prompt1 = ChatPromptTemplate.from_template("{genre} 장르에서 추천할 만한 영화를 한 편 알려주세요.")

# Step 2: 추천된 영화의 줄거리를 요약
prompt2 = ChatPromptTemplate.from_template("{movie} 추전한 영화의 제목을 먼저 알려주시고, 줄을 바꾸어서 영화의 줄거리를 7문장으로 요약해 주세요.")

# OpenAI 모델 사용
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

# 체인 1: 영화 추천 (입력: 장르 → 출력: 영화 제목)
chain1 = prompt1 | llm | StrOutputParser()

In [19]:
# 체인 2: 줄거리 요약 (입력: 영화 제목 → 출력: 줄거리)
chain2 = (
    {"movie": chain1}  # chain1의 출력을 movie 변수로 전달
    | prompt2
    | llm
    | StrOutputParser()
)

# 실행: "SF" 장르의 영화 추천 및 줄거리 요약
response = chain2.invoke({"genre": "액션"})
print(response)  

'매드 맥스: 분노의 도로'

- 줄거리 요약 -

맥스는 포스트 아포칼립스 세계에서 자신의 차를 빼앗긴 악당들과의 전투 속에서 미친 듯이 달리며 쫓기게 된다. 그리고 그는 무모한 여성 전사인 퍼리오사와 만나 함께 악당들을 상대로 사상 최악의 전투를 벌이게 된다. 맥스는 물자가 부족한 세상에서 살아가기 위해 불타는 소목에 직면하게 되지만, 그는 자신의 용기와 전투력으로 어려움을 극복해 나가며 포스트 아포칼립스 세계에서 살아남기 위한 모험을 떠난다. 퍼리오사와 함께한 맥스는 악당들과의 치열한 싸움 속에서 삶과 죽음을 건 결전을 펼치게 되는데, 그 과정에서 맥스는 내면의 분노와 욕망, 그리고 소망과 희망을 마주하게 되는데...


### PromptTemplate 여러개 연결하기

In [ ]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# 템플릿에 값을 채워서 프롬프트를 완성
filled_prompt = prompt_template.format(model_name="ChatGPT", count=3)

# 문자열 템플릿 결합 (PromptTemplate + PromptTemplate + 문자열)
combined_prompt = (
              prompt_template
              + PromptTemplate.from_template("\n\n 그리고 {model_name} 모델의 장점을 요약 정리해 주세요")
              + "\n\n {model_name} 모델과 비슷한 AI 모델은 어떤 것이 있나요? 모델명은 {language}로 답변해 주세요."
)
combined_prompt.format(model_name="ChatGPT", count=3, language="영어")

llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
chain = combined_prompt | llm | StrOutputParser()
response = chain.invoke({"model_name":"ChatGPT", "count":3, "language":"영어"})


### 여러개의 PromptTemplate 만들기

In [ ]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

questions = [
    {"model_name": "GPT-4", "count": 2},
    {"model_name": "Gemma", "count": 3},
]

# 여러 개의 프롬프트를 미리 생성
formatted_prompts = [prompt_template.format(**q) for q in questions]
print(formatted_prompts)  # 미리 생성된 질문 목록 확인

['GPT-4 모델의 학습 원리를 2 문장으로 한국어로 답변해 주세요.', 'Gemma 모델의 학습 원리를 3 문장으로 한국어로 답변해 주세요.']


In [ ]:
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

for prompt in formatted_prompts:
    response = llm.invoke(prompt)
    print(response.content)

content='GPT-4 모델은 대규모의 데이터셋을 사용하여 자기 지도 학습 방식으로 사전 훈련되며, 텍스트의 다음 단어를 예측하는 과정을 통해 문맥을 이해하고 생성합니다. 이 모델은 엄청난 양의 매개변수와 계산 능력을 가지고 있어 자연어 처리 작업에서 뛰어난 성능을 발휘합니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 132, 'prompt_tokens': 40, 'total_tokens': 172, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BgnWMYdOyniIK1zt4Q7RMrjJXUZCH', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--3df5e478-3a58-4cdf-8a18-4a4e88ae1a13-0' usage_metadata={'input_tokens': 40, 'output_tokens': 132, 'total_tokens': 172, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
content='Gemma 모델은 전이 학습을 통해 기존의 모델을 사용하여 새로운 데이터셋에 대한 학습 성능을 향상시킵니다. Gemma 모델

### ChatPromptTemplate
- SystemMessagePromptTemplate, HumanMessagePromptTemplate, 
AIMessagePromptTemplate 

In [29]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate
)
from langchain_openai import ChatOpenAI

# 개별 메시지 템플릿 정의
system_message = SystemMessagePromptTemplate.from_template(
    "You are an {topic} expert. Please provide clear and detailed explanations."
)
user_message = HumanMessagePromptTemplate.from_template(
    "{question}"
)
ai_message = AIMessagePromptTemplate.from_template(
    "This is an example answer about {topic}."
)

# ChatPromptTemplate로 메시지들을 묶기
chat_prompt = ChatPromptTemplate.from_messages([
    system_message,
    user_message,
    ai_message
])

# 메시지 생성
messages = chat_prompt.format_messages(topic="AI", question="What is deep learning?")

# LLM 호출
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
response = llm.invoke(messages)

# 결과 출력
print(response.content)

Deep learning is a subfield of machine learning that involves building and training artificial neural networks with multiple layers (hence the term "deep"). These neural networks are designed to automatically learn and represent complex patterns and relationships within data. Deep learning has seen significant advancements in recent years, particularly with the use of deep neural networks known as convolutional neural networks (CNNs) for image recognition, recurrent neural networks (RNNs) for sequential data processing, and transformers for natural language processing tasks. Deep learning has been applied successfully in various domains such as computer vision, speech recognition, natural language processing, and reinforcement learning. Its ability to automatically learn hierarchical representations from data has led to improved performance in tasks that were previously considered challenging for traditional machine learning algorithms.
